### install dependencies
only do this once

In [1]:
#! python3 -m venv venv
#! source venv/bin/activate
#! pip3 install -r requirements.txt
#! pip install ipykernel
#! python -m ipykernel install --user
#! pip install -e ./
#! pip freeze

### imports

In [2]:
from builtins import range
import numpy as np
import pandas as pd
import os
import logging

In [3]:
from importlib import reload
from src.embedding_utilities import euclidean_dist, cosine_dist
from src.embedding import Embedding


In [4]:
logging.basicConfig(encoding='utf-8', level=logging.INFO)

# define paths

In [5]:
#s3_client = boto3.client('s3')
BUCKET_NAME = 'ma-2021-07-word-embeddings'
FILE_NAME_EN = 'glove.6B.300d.txt'
FILE_NAME_DE = 'GloVe_vectors_de_10_test_file.txt '#'GloVe_vectors_de_50000.txt'

In [61]:
glove_path_en = ['../data/glove.6B.100d.txt']
#glove_path_de = ['../data/GloVe_vectors_de_part_01.txt', '../data/GloVe_vectors_de_part_02.txt']
glove_path_de = ['../data/GloVe_vectors_de_50000.txt']#['../data/GloVe_vectors_de.txt']

### the cosine distance does not change when doubeling
cosine distance only takes into account angle (the smaller the higher the similarity.)

euclidian looks at absolute length

In [62]:
a = pd.Series([1,1])
b = pd.Series([1,2])
print(f'euclidian distance {euclidean_dist(a,b)}')
print(f'cosine distance {cosine_dist(a,b)}')

euclidian distance 1.0
cosine distance 0.05131670194948623


In [63]:
a_double = a*2
b_double = b*2
print(f'euclidian distance {euclidean_dist(a_double,b_double)}')
print(f'cosine distance {cosine_dist(a_double,b_double)}')

euclidian distance 2.0
cosine distance 0.05131670194948623


In [64]:
dist, metric = cosine_dist, 'cosine'
#alternative: dist, metric = dist1, 'euclidean'

### load pretrained word vectors
source: https://www.kaggle.com/anindya2906/glove6b

In [65]:
import s3fs
import csv

In [66]:
%time glove_embedding_en = Embedding(language='en', path_list=glove_path_en)

INFO:root:loading word embeddings word to vec from path ['../data/glove.6B.100d.txt']
/Users/magdalena.aretz/code/nlp_scripts/venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column 0 - only the converter will be used
  return func(*args, **kwargs)
INFO:root:read in embeddings of shape (399997, 100)
INFO:root:read in  word2vec
INFO:root:read in index_to_word


total number of entries found:  399997. Dimension: 100
CPU times: user 42.1 s, sys: 1.66 s, total: 43.7 s
Wall time: 43.8 s


In [67]:
glove_embedding_en.embedding.head(2)

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
the,-0.038208,-0.244873,0.728027,-0.399658,0.083191,0.043945,-0.391357,0.334473,-0.575684,0.087463,...,0.016220,-0.017105,-0.389893,0.874023,-0.725586,-0.510742,-0.520508,-0.145874,0.827637,0.270508
",",-0.107666,0.110535,0.598145,-0.543457,0.673828,0.106628,0.038879,0.354736,0.063538,-0.094177,...,0.349609,-0.722656,0.375488,0.444092,-0.990723,0.612305,-0.351074,-0.831543,0.452881,0.082581


In [68]:
# working but incredible slow:
# lokal: ca 10 sec for 400k
# read from aws s3: ca 40 sek for 4k => 4000 sek für 400k => faktor 400 schlechter
#%time glove_embedding_en = Embedding(language='en', path=f's3://{BUCKET_NAME}/{FILE_NAME_EN}')

In [69]:
print(glove_embedding_en.embedding.shape)
glove_embedding_en.embedding.head(1)

(399997, 100)


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
the,-0.038208,-0.244873,0.728027,-0.399658,0.083191,0.043945,-0.391357,0.334473,-0.575684,0.087463,...,0.01622,-0.017105,-0.389893,0.874023,-0.725586,-0.510742,-0.520508,-0.145874,0.827637,0.270508


In [70]:
#1.3 Mio entries => takes about 1000sec on local machine 1309281, 360 secs after
#50k entries => takes 9 sec on local machine before, 6 secs after

#50k entries => takes about 9 mins when reading linewise from aws

# aws
#%time glove_embedding_de = Embedding(language='de', path_list = [f's3://{BUCKET_NAME}/{FILE_NAME_DE}'])

#lokal
%time glove_embedding_de = Embedding(language='de', path_list=glove_path_de)

INFO:root:loading word embeddings word to vec from path ['../data/GloVe_vectors_de_50000.txt']
/Users/magdalena.aretz/code/nlp_scripts/venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column 0 - only the converter will be used
  return func(*args, **kwargs)
INFO:root:read in embeddings of shape (46994, 300)
INFO:root:read in  word2vec
INFO:root:read in index_to_word


total number of entries found:  46994. Dimension: 300
CPU times: user 14 s, sys: 414 ms, total: 14.4 s
Wall time: 14.4 s


In [71]:
print(glove_embedding_de.embedding.shape)
glove_embedding_de.embedding.head(2)

(46994, 300)


,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
starke,-0.285156,-0.350098,0.066101,-0.059021,-0.074280,0.105103,-0.091431,-0.379395,-0.471924,-0.038055,...,-0.046783,0.486572,0.092285,0.394775,-0.386230,0.016998,0.315918,0.434814,-0.198242,-0.065308
mittagszeit,0.402832,-0.000529,-0.063110,-0.429688,0.059174,-0.181274,0.278076,-0.885254,-0.528809,0.113342,...,-0.310059,-0.564453,-0.512207,-0.061523,-0.261719,0.100830,0.032684,0.089478,0.310791,0.315674


In [72]:
# dict: word => embedding
print(type(glove_embedding_en.word2vec))
print(len(glove_embedding_en.word2vec))
print(len(glove_embedding_en.word2vec['tree']))
print(glove_embedding_en.word2vec['tree'])

<class 'dict'>
399997
100
{1: -0.9501953125, 2: 1.1875, 3: -0.1558837890625, 4: -0.07403564453125, 5: 0.5078125, 6: 0.31591796875, 7: -0.548828125, 8: 0.06732177734375, 9: 0.15185546875, 10: -0.55126953125, 11: 0.239013671875, 12: 0.517578125, 13: 0.74658203125, 14: 0.250732421875, 15: 0.167236328125, 16: 0.623046875, 17: 0.578125, 18: 0.5126953125, 19: 0.40380859375, 20: 0.287353515625, 21: -0.86474609375, 22: 0.308349609375, 23: 0.5107421875, 24: 0.234619140625, 25: 0.79345703125, 26: 1.0947265625, 27: -0.38232421875, 28: -0.615234375, 29: -0.317626953125, 30: 0.0576171875, 31: -0.062225341796875, 32: 0.2337646484375, 33: 1.3642578125, 34: 0.041259765625, 35: 0.161376953125, 36: 0.5126953125, 37: -0.59228515625, 38: 0.1998291015625, 39: 0.84033203125, 40: -0.29443359375, 41: -0.60302734375, 42: -0.40185546875, 43: 0.2493896484375, 44: 0.1168212890625, 45: 0.320068359375, 46: 0.475341796875, 47: 0.06585693359375, 48: 0.82470703125, 49: -0.44140625, 50: -0.2509765625, 51: -0.662109375,

In [73]:
# list: index => word
print(type(glove_embedding_en.index_to_word))
print(glove_embedding_en.index_to_word[1])
print(glove_embedding_de.index_to_word[1])

<class 'list'>
,
mittagszeit


In [74]:
print(glove_embedding_en.embedding.shape)
print(glove_embedding_de.embedding.shape)
print (glove_embedding_en.embedding.head(2))

(399997, 100)
(46994, 300)
          1         2         3         4         5         6         7    \
0                                                                           
the -0.038208 -0.244873  0.728027 -0.399658  0.083191  0.043945 -0.391357   
,   -0.107666  0.110535  0.598145 -0.543457  0.673828  0.106628  0.038879   

          8         9         10   ...       91        92        93   \
0                                  ...                                 
the  0.334473 -0.575684  0.087463  ...  0.016220 -0.017105 -0.389893   
,    0.354736  0.063538 -0.094177  ...  0.349609 -0.722656  0.375488   

          94        95        96        97        98        99        100  
0                                                                          
the  0.874023 -0.725586 -0.510742 -0.520508 -0.145874  0.827637  0.270508  
,    0.444092 -0.990723  0.612305 -0.351074 -0.831543  0.452881  0.082581  

[2 rows x 100 columns]


In [75]:
import sys
# approx size in MB sys.getsizeof() on the other hand returns the memory size of the object:
print(sys.getsizeof(glove_embedding_en.word2vec)//1000000)
print(sys.getsizeof(glove_embedding_en.index_to_word)//1000000)
print(sys.getsizeof(glove_embedding_en.embedding)//1000000)

20
3
114


In [107]:
print(glove_embedding_en.embedding.info(verbose=False, memory_usage="deep"))
print(glove_embedding_en.embedding.min().min())
print(glove_embedding_en.embedding.max().max())

<class 'pandas.core.frame.DataFrame'>
Index: 399997 entries, the to sandberger
Columns: 100 entries, 1 to 100
dtypes: float16(100)
memory usage: 109.2 MB
None
-4.066
3.574


In [108]:
print(glove_embedding_de.embedding.info(verbose=False, memory_usage="deep"))
print(glove_embedding_de.embedding.min().min())
print(glove_embedding_de.embedding.max().max())

<class 'pandas.core.frame.DataFrame'>
Index: 46994 entries, starke to pressekonferenz
Columns: 300 entries, 1 to 300
dtypes: float16(300)
memory usage: 31.0 MB
None
-3.932
4.816


# Best of

In [77]:
glove_embedding_en.find_analogies('man', 'fun', 'woman')

INFO:root:man - fun =  woman - kids


'kids'

In [78]:
glove_embedding_de.find_analogies('mann', 'spass', 'frau')

INFO:root:mann - spass =  frau - schwiegermutter


'schwiegermutter'

In [24]:
glove_embedding_en.find_analogies('woman', 'love', 'man')

INFO:root:woman - love =  man - me


'me'

In [25]:
glove_embedding_de.find_analogies('frau', 'liebe', 'mann')

INFO:root:frau - liebe =  mann - hickey


'hickey'

In [26]:
glove_embedding_en.find_analogies('man', 'motivation', 'woman')

INFO:root:man - motivation =  woman - satisfaction


'satisfaction'

In [27]:
glove_embedding_de.find_analogies('mann', 'motivation', 'frau')

INFO:root:mann - motivation =  frau - beeinflussen


'beeinflussen'

In [28]:
glove_embedding_en.find_analogies('woman', 'motivation', 'man')

INFO:root:woman - motivation =  man - tremendous


'tremendous'

In [79]:
glove_embedding_de.find_analogies('frau', 'motivation', 'mann')

INFO:root:frau - motivation =  mann - iranischen


'iranischen'

In [80]:
glove_embedding_en.find_analogies('man', 'respect', 'woman')

INFO:root:man - respect =  woman - dignity


'dignity'

In [81]:
glove_embedding_de.find_analogies('mann', 'respekt', 'frau')

INFO:root:mann - respekt =  frau - verschroben


'verschroben'

In [82]:
glove_embedding_en.find_analogies('man', 'success', 'woman')
glove_embedding_de.find_analogies('mann', 'erfolg', 'woman')

INFO:root:man - success =  woman - achieved
INFO:root:mann - erfolg =  woman - lia


'lia'

In [83]:
glove_embedding_en.find_analogies('man', 'love', 'woman')
glove_embedding_de.find_analogies('mann', 'liebe', 'frau')

INFO:root:man - love =  woman - mother
INFO:root:mann - liebe =  frau - hickey


'hickey'

In [84]:
glove_embedding_en.find_analogies('woman', 'mother in law', 'man')
glove_embedding_de.find_analogies('frau', 'schwiegermutter', 'mann')
glove_embedding_de.find_analogies('mann', 'schwiegervater', 'frau')

INFO:root:frau - schwiegermutter =  mann - iranischen


mother in law not in dictionary


INFO:root:mann - schwiegervater =  frau - untergräbt


'untergräbt'

In [85]:
glove_embedding_en.find_analogies('man', 'great', 'woman')
glove_embedding_de.find_analogies('mann', 'großartig', 'frau')

INFO:root:man - great =  woman - little
INFO:root:mann - großartig =  frau - langweilig


'langweilig'

In [86]:
glove_embedding_en.find_analogies('women', 'kids', 'man')
glove_embedding_de.find_analogies('frau', 'kinder', 'mann')

INFO:root:women - kids =  man - kid
INFO:root:frau - kinder =  mann - iranischen


'iranischen'

In [87]:
glove_embedding_en.find_analogies('man', 'friends', 'woman')
glove_embedding_de.find_analogies('mann', 'freunde', 'frau')

INFO:root:man - friends =  woman - parents
INFO:root:mann - freunde =  frau - suggeriert


'suggeriert'

In [88]:
glove_embedding_en.find_analogies('men', 'friends', 'women')
glove_embedding_de.find_analogies('männer', 'freunde', 'frauen')

INFO:root:men - friends =  women - parents
INFO:root:männer - freunde =  frauen - ihrer


'ihrer'

In [89]:
glove_embedding_en.find_analogies('woman', 'safety', 'man')
glove_embedding_de.find_analogies('frau', 'sicherheit', 'mann')

INFO:root:woman - safety =  man - security
INFO:root:frau - sicherheit =  mann - ausrüstung


'ausrüstung'

In [90]:
glove_embedding_en.find_analogies('women', 'orgasm', 'men')
glove_embedding_de.find_analogies('frauen', 'orgasmus', 'männer')

INFO:root:women - orgasm =  men - 551
INFO:root:frauen - orgasmus =  männer - kleidern


'kleidern'

In [91]:
glove_embedding_en.find_analogies('woman', 'money', 'man')
glove_embedding_de.find_analogies('frau', 'geld', 'mann')

INFO:root:woman - money =  man - cash
INFO:root:frau - geld =  mann - iranischen


'iranischen'

In [92]:
# klappt einfach
glove_embedding_en.find_analogies('woman', 'man', 'she')
glove_embedding_en.find_analogies('woman', 'man', 'aunt')
glove_embedding_en.find_analogies('woman', 'man', 'sister')
glove_embedding_en.find_analogies('woman', 'man', 'actress')
glove_embedding_en.find_analogies('woman', 'man', 'mother')

INFO:root:woman - man =  she - he
INFO:root:woman - man =  aunt - uncle
INFO:root:woman - man =  sister - brother
INFO:root:woman - man =  actress - actor
INFO:root:woman - man =  mother - father


'father'

In [93]:
# klappt einfach
glove_embedding_de.find_analogies('frau', 'mann', 'sie')
glove_embedding_de.find_analogies('frau', 'mann', 'tante')
glove_embedding_de.find_analogies('frau', 'mann', 'schwester')
glove_embedding_de.find_analogies('frau', 'mann', 'schauspielerin')
glove_embedding_de.find_analogies('frau', 'mann', 'mutter')

INFO:root:frau - mann =  sie - göttliche
INFO:root:frau - mann =  tante - iranischen
INFO:root:frau - mann =  schwester - iranischen
INFO:root:frau - mann =  schauspielerin - schauspieler
INFO:root:frau - mann =  mutter - iranischen


'iranischen'

In [94]:
#klappt einfach
glove_embedding_en.find_analogies('paris', 'france', 'london')
glove_embedding_en.find_analogies('paris', 'france', 'rome')
glove_embedding_en.find_analogies('france', 'paris', 'italy')
glove_embedding_en.find_analogies('florida', 'miami', 'texas')

INFO:root:paris - france =  london - britain
INFO:root:paris - france =  rome - italy
INFO:root:france - paris =  italy - rome
INFO:root:florida - miami =  texas - dallas


'dallas'

In [97]:
glove_embedding_de.find_analogies('frankreich', 'paris', 'england')
glove_embedding_de.find_analogies('frankreich', 'paris', 'italien')
glove_embedding_de.find_analogies('paris', 'frankreich', 'rom')
glove_embedding_de.find_analogies('miami', 'florida', 'bayern')

INFO:root:frankreich - paris =  england - london
INFO:root:frankreich - paris =  italien - leihmutter
INFO:root:paris - frankreich =  rom - hymne
INFO:root:miami - florida =  bayern - landkreis


'landkreis'

In [98]:
#klappt einfach
glove_embedding_en.find_analogies('november', 'december', 'march')
glove_embedding_de.find_analogies('november', 'dezember', 'märz')

INFO:root:november - december =  march - july
INFO:root:november - dezember =  märz - indianern


'indianern'

In [99]:
#klappt einfach
glove_embedding_en.find_analogies('einstein', 'scientist', 'dali')
glove_embedding_de.find_analogies('einstein', 'wissenschaftler', 'merkel')

INFO:root:einstein - scientist =  dali - cumin


merkel not in dictionary


In [100]:
# fun
glove_embedding_en.find_analogies('china', 'rice', 'germany')
glove_embedding_de.find_analogies('china', 'reis', 'deutschland')

INFO:root:china - rice =  germany - fischer
INFO:root:china - reis =  deutschland - anzeige


'anzeige'

In [101]:
glove_embedding_en.find_analogies('germany', 'beer', 'usa')
glove_embedding_de.find_analogies('deutschland', 'bier', 'usa')

INFO:root:germany - beer =  usa - liquor
INFO:root:deutschland - bier =  usa - gebraut


'gebraut'

In [102]:
glove_embedding_en.find_analogies('usa', 'freedom', 'germany')
glove_embedding_de.find_analogies('usa', 'freiheit', 'deutschland')

INFO:root:usa - freedom =  germany - germans
INFO:root:usa - freiheit =  deutschland - anzeige


'anzeige'

In [103]:
glove_embedding_en.find_analogies('builder', 'building', 'politician')
glove_embedding_en.find_analogies('writer', 'book', 'politician')
glove_embedding_de.find_analogies('bauarbeiter', 'baustelle', 'politiker')
glove_embedding_de.find_analogies('schriftsteller', 'buch', 'politiker')

INFO:root:builder - building =  politician - political
INFO:root:writer - book =  politician - election
INFO:root:bauarbeiter - baustelle =  politiker - jurist
INFO:root:schriftsteller - buch =  politiker - mitglied


'mitglied'

In [104]:
glove_embedding_en.find_analogies('hacker', 'code', 'politician')
glove_embedding_de.find_analogies('hacker', 'code', 'politiker')

INFO:root:hacker - code =  politician - constitution
INFO:root:hacker - code =  politiker - lilie


'lilie'

In [105]:
glove_embedding_en.find_analogies('trump', 'democracy', 'hitler')
glove_embedding_de.find_analogies('trump', 'demokratie', 'hitler')

INFO:root:trump - democracy =  hitler - communist
INFO:root:trump - demokratie =  hitler - gerungen


'gerungen'

In [106]:
glove_embedding_de.get_embedding('trump')

{1: 0.1866455078125,
 2: 0.59521484375,
 3: 0.0906982421875,
 4: -0.44287109375,
 5: 0.0018529891967773438,
 6: -0.1324462890625,
 7: 0.2242431640625,
 8: 0.10614013671875,
 9: -0.41650390625,
 10: -0.6142578125,
 11: -0.80322265625,
 12: 0.54443359375,
 13: -0.48291015625,
 14: 0.03424072265625,
 15: -0.09881591796875,
 16: 0.357421875,
 17: -0.05462646484375,
 18: -0.049346923828125,
 19: 0.28955078125,
 20: -0.03887939453125,
 21: 0.057708740234375,
 22: 0.5888671875,
 23: 0.182373046875,
 24: -0.07330322265625,
 25: 0.0943603515625,
 26: 0.286865234375,
 27: -0.5791015625,
 28: 0.861328125,
 29: -0.275634765625,
 30: -0.67822265625,
 31: 0.301513671875,
 32: -0.5322265625,
 33: 0.437744140625,
 34: 0.33154296875,
 35: -0.2203369140625,
 36: -0.0011091232299804688,
 37: 0.1834716796875,
 38: -0.411865234375,
 39: 0.56982421875,
 40: 0.337158203125,
 41: -0.282470703125,
 42: -0.2529296875,
 43: 0.0426025390625,
 44: -0.32763671875,
 45: -0.06982421875,
 46: -0.5966796875,
 47: 0.419

# nearest neighbors

In [55]:
glove_embedding_en.nearest_neighbors('france')

AttributeError: 'dict' object has no attribute 'reshape'

In [ ]:
glove_embedding_de.nearest_neighbors('frankreich')

In [ ]:
glove_embedding_en.nearest_neighbors('einstein')
glove_embedding_de.nearest_neighbors('einstein')

In [ ]:
glove_embedding_en.nearest_neighbors('trump')
glove_embedding_de.nearest_neighbors('trump')

In [ ]:
glove_embedding_en.nearest_neighbors('merkel')
glove_embedding_de.nearest_neighbors('merkel')

In [ ]:
glove_embedding_en.nearest_neighbors('procrastination')
glove_embedding_de.nearest_neighbors('prokrastination')

In [ ]:
glove_embedding_en.nearest_neighbors('religion')
glove_embedding_de.nearest_neighbors('religion')

In [ ]:
nearest_neighbors('woman',index_to_word, word2vec)
nearest_neighbors('man',index_to_word, word2vec)
nearest_neighbors('nephew',index_to_word, word2vec)

In [ ]:
nearest_neighbors('february',index_to_word, word2vec)
nearest_neighbors('success',index_to_word, word2vec)
nearest_neighbors('money',index_to_word, word2vec)
nearest_neighbors('love',index_to_word, word2vec)

In [ ]:
nearest_neighbors('procrastination', index_to_word, word2vec)